```python
def sha1(data: bytes) -> bytes
def sha256(data: bytes) -> bytes
def sha512(data: bytes) -> bytes
def ripemd160(data: bytes) -> bytes
def recover_key(data: bytes) -> bytes

def assert_sha1(data: bytes, hash: bytes)
def assert_sha256(data: bytes, hash: bytes)
def assert_sha512(data: bytes, hash: bytes)
def assert_ripemd160(data: bytes, hash: bytes)
def assert_recover_key(data: bytes, hash: bytes)
```

In [1]:
from init import *

In [2]:
code = '''
import chain
def apply(receiver, code, action):
    print(chain.sha1(b'helloworld'))
    chain.assert_sha1(b'helloworld', chain.sha1(b'helloworld'))
    chain.assert_sha256(b'helloworld', chain.sha256(b'helloworld'))
    chain.assert_sha512(b'helloworld', chain.sha512(b'helloworld'))
    chain.assert_ripemd160(b'helloworld', chain.ripemd160(b'helloworld'))
'''

code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)
r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


b'j\xdf\xb1\x83\xa4\xa2\xc9J/\x92\xda\xb5\xad\xe7b\xa4x\x89\xa5\xa1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [4]:
uuosapi.get_account(python_contract)

{'account_name': 'ceyelqpjeeia',
 'head_block_num': 44640296,
 'head_block_time': '2021-01-07T06:35:29.500',
 'privileged': False,
 'last_code_update': '2021-01-07T06:02:15.000',
 'created': '2020-11-30T02:20:16.000',
 'core_liquid_balance': '78.8595 TNT',
 'ram_quota': 3810651,
 'net_weight': 30000,
 'cpu_weight': 310000,
 'net_limit': {'used': 611100, 'available': 6577645, 'max': 7188745},
 'cpu_limit': {'used': 173620, 'available': 13993511, 'max': 14167131},
 'ram_usage': 2732153,
 'permissions': [{'perm_name': 'active',
   'parent': 'owner',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS8moVJwVaR8pb43KBi2HwmtbwebXmjSJUerfwbvFW41tEvYsfDE',
      'weight': 1}],
    'accounts': [{'permission': {'actor': 'ceyelqpjeeia',
       'permission': 'eosio.code'},
      'weight': 1}],
    'waits': []}},
  {'perm_name': 'owner',
   'parent': '',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS79fssH8wH5EyNtKP2oZAt1b9jfeu6hpna7RJ7eMwwjr73S5kWa',
      'weight': 1}],

In [ ]:
b'EOS8moVJwVaR8pb43KBi2HwmtbwebXmjSJUerfwbvFW41tEvYsfDE

In [5]:
import base58
public_key = 'EOS8moVJwVaR8pb43KBi2HwmtbwebXmjSJUerfwbvFW41tEvYsfDE'

print(public_key)
b'\x00'+base58.b58decode(public_key[3:])

EOS8moVJwVaR8pb43KBi2HwmtbwebXmjSJUerfwbvFW41tEvYsfDE


b'\x00\x03\xff\x99\x99]\x1e\xc7\x9evb\xbc\x7f\xfa\x07j\xec\xed\xa0\x0f\xb9T\x04\x06\t\x1a\xd3k\xf8\xf2\xecX\xd6Q\xfc\xac\x0b\xbd'

```python
def assert_recover_key(digest, sign, public_key) -> None
def recover_key(digest, sign) -> bytes
```

In [13]:

code = r'''
import struct
import chain
public_key = b'\x00\x03\xff\x99\x99]\x1e\xc7\x9evb\xbc\x7f\xfa\x07j\xec\xed\xa0\x0f\xb9T\x04\x06\t\x1a\xd3k\xf8\xf2\xecX\xd6Q\xfc\xac\x0b\xbd'

def apply(receiver, code, action):
    data = chain.read_action_data()
    value, size = struct.unpack_uint32(data)
    text = data[size:value+size]
    sign = data[size+value:]
    print(text)
    print(data)
    print(len(public_key))

    digest = chain.sha256(text)
    chain.assert_recover_key(digest, sign, public_key)
    
    key = chain.recover_key(digest, sign)
    print(key)
    print(len(key))
    
    checksum = chain.ripemd160(key[1:])
    print(checksum)
'''
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"text", "type":"bytes"},
        {"name":"sign", "type":"signature"}
        ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''


uuosapi.set_abi(python_contract, abi)

code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)


text = b'hello,world'
h = hashlib.sha256()
h.update(text)
signature = wallet.sign_digest(h.digest(), 'EOS8moVJwVaR8pb43KBi2HwmtbwebXmjSJUerfwbvFW41tEvYsfDE')

args = {'text': text.hex(), 'sign': signature}

r = uuosapi.push_action(python_contract, 'sayhello', args, {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])

b'hello,world'
b'\x0bhello,world\x00 z\xc3}P\x07\x81\xc4[qf\x82\xa71\xe8\xa3\xd0\x12p\xfcHs\xc2\xb2\xb9\xee\x19\x8ePr\x03\x9d\xe3\x12\x99\x16\xd3HW\x80K\xb9\xa2\x14\x93\x0e9Cf\xf7-\xb2\x91\xfe\x9f\x80q\xa2\xbf\xd7n\x06\xd7.S'
38
b'\x00\x03\xff\x99\x99]\x1e\xc7\x9evb\xbc\x7f\xfa\x07j\xec\xed\xa0\x0f\xb9T\x04\x06\t\x1a\xd3k\xf8\xf2\xecX\xd6Q'
34
b'\xfc\xac\x0b\xbd\xc1X\x82\xa0\x98D\xf2\x10\xe5\xd8y\x81YN\xa0*\xb9T\x04\x06\x01\x00\x00\x00\x01\x00\x00\x00'



In [17]:
import base58
public_key = 'EOS8moVJwVaR8pb43KBi2HwmtbwebXmjSJUerfwbvFW41tEvYsfDE'
print(public_key)
raw_pub_key = base58.b58decode(public_key[3:])[:-4]
print(raw_pub_key)

EOS8moVJwVaR8pb43KBi2HwmtbwebXmjSJUerfwbvFW41tEvYsfDE
b'\x03\xff\x99\x99]\x1e\xc7\x9evb\xbc\x7f\xfa\x07j\xec\xed\xa0\x0f\xb9T\x04\x06\t\x1a\xd3k\xf8\xf2\xecX\xd6Q'


In [18]:
import base58
h = hashlib.new('ripemd160')
h.update(raw_pub_key)

digest = h.digest()

public_key = raw_pub_key + digest[:4]
b'EOS'+base58.b58encode(public_key)


b'EOS8moVJwVaR8pb43KBi2HwmtbwebXmjSJUerfwbvFW41tEvYsfDE'